In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("code")
import numpy as np
import matplotlib.pyplot as plt
from keras_tqdm import TQDMNotebookCallback
from npm3dGenerator_tree import NPM3DGenerator, NPM3DGenerator_full
from models import build_point_net, save_model, load_model
import utils_training as u_t

Using TensorFlow backend.


In [ ]:
training_dataset = NPM3DGenerator_full(paths_to_keep = [1], use_normals = True)

../Benchmark/training_10_classes_subsampled_2\Lille1_2_tree.joblib
COMPUTING TREE
DONE
MODIFIYING PATHS ../Benchmark/training_10_classes_subsampled_2\Lille1_2.ply
COMPUTING NORMALS
SPLITTING 57


In [ ]:
training_dataset.class_weight

In [ ]:
miou_metric = u_t.mIoU(training_dataset.n_classes)

model = build_point_net(input_shape = (training_dataset.n_points, training_dataset.n_channels),
                        output_shape = training_dataset.n_classes)
model.compile("adam", u_t.my_categorical_crossentropy, metrics = ["accuracy", miou_metric.miou])
#model.summary()

In [ ]:
n_epochs = 1
history = model.fit_generator(training_dataset, epochs = n_epochs,
                              validation_data = training_dataset,
                              class_weight = training_dataset.class_weight,
                              verbose = 0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
#save_model(model, "trained_{}_{}_{}".format(training_dataset.n_points, training_dataset.n_channels, training_dataset.n_classes))
u_t.plot_history(history)

In [ ]:
test_dataset = NPM3DGenerator(input_dir = "../Benchmark_MVA/test", train = False)
model = load_model("trained_{}_{}_{}".format(test_dataset.n_points, test_dataset.n_channels, test_dataset.n_classes))
predictions = test_dataset.predict_point_cloud(model)
plt.hist(np.argmax(predictions, axis = -1))